In [12]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids, and separate the dataset:

In [13]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [14]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [15]:
# We Separate the dataset
from implementations import *

tX_list, ids_list, y_list = separate_dataset(tX, ids, y)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test) 

In [11]:
test = np.array([1,100,1])
print(logsig(test))

[0 0 0]


## Grid Search

In [44]:
function = ridge_regression
degree_vec = []
lambda_vec = []
for i in range(3):
    print('Dataset part {l}:'.format(l = i))
    rmse_te, BestDeg, BestLambda = grid_search(y_list[i], tX_list[i], function, True, degrees= range(1, 10))
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    print('     Best degree: {d}'.format(d = BestDeg))
    print('     Best lambda: {m}'.format(m = BestLambda))
    print('     Loss: {lo}'.format(lo = rmse_te))

Dataset part 0:
     Best degree: 1
     Best lambda: 0
     Loss: 0.717038664726091
Dataset part 1:
     Best degree: 3
     Best lambda: 0
     Loss: 0.8061285859414737
Dataset part 2:
     Best degree: 5
     Best lambda: 0
     Loss: 0.759861504623161


In [45]:
weights_list = []
loss_list = []
mat_tX_test_list = []

for i in range(3):
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], True, tX_test_list[i])
    w, l = ridge_regression(y_list[i], mat_tX, lambda_vec[i])
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)
    

## Gradient Descent

In [ ]:
# Linear regression using gradient descent

initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 3e-7

weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)

## Least squares

In [76]:
# Least squares regression using normal equation
weights, loss = least_squares(y, tX)
loss

0.3396868094770349

## Ridge regression

In [9]:
# Ridge regression using normal equations
lambda_ = 1e-3
weights, loss = ridge_regression(y, tX, lambda_)
loss

(30,)

In [59]:
# Ridge regression with data augmentation
lambda_ = 1e-3
mat_tX = build_poly(tX, 5)
weights, loss = ridge_regression(y, mat_tX, lambda_)

loss

0.2967432202164457

In [35]:
# Ridge regression with separated data
lambda_ = 1e-3
weights_list, loss_list = separated_train(tX_list, y_list, ridge_regression, lambda_)

In [76]:
# Ridge regression on separated datset with data augmentation
lambda_ = 1e-3
mat_tX_list = build_poly_separated(tX_list, 3)
# weights, loss = ridge_regression(y_list[0], mat_tX_list[0], lambda_)

weights_list, loss_list = separated_train(mat_tX_list, y_list, ridge_regression, lambda_)

## Logistic regression

In [37]:
weights_list = []
max_iters = 1500
gamma = 1e-4
mat_tX_test_list = []

for i in range(3):
    mat_tX_list = build_poly(tX_list[i], 1)
    initial_w = np.zeros((mat_tX_list.shape[1], 1))
    w, _ = logistic_regression(np.where(y_list[i] == -1, 0, y_list[i]), mat_tX_list, initial_w, max_iters, gamma)
    weights_list.append(w)
    mat_tX_test_list.append(tX_test_list[i])



Current iteration=0, loss=69254.41425128403
Current iteration=100, loss=81449.71895804847
Current iteration=200, loss=80696.63673349045
Current iteration=300, loss=80592.32343246232
Current iteration=400, loss=80576.40882777364
Current iteration=500, loss=80573.89748207579
Current iteration=600, loss=80573.49583033338
Current iteration=700, loss=80573.43125526051
Current iteration=800, loss=80573.42085238648
Current iteration=900, loss=80573.41917522243
Current iteration=1000, loss=80573.4189047484
Current iteration=1100, loss=80573.41886112426
Current iteration=1200, loss=80573.41885408778
Current iteration=1300, loss=80573.41885295244


KeyboardInterrupt: 

In [20]:
#Log Reg Pen


degree_vec = [1]
lambda_vec = [1e-05]
gamma_vec = [1e-2]

weights_list = []
loss_list = []
mat_tX_test_list = []
max_iter= 100

for i in range(3):
    print("loop")
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[0], False, tX_test_list[i])
    initial_w = np.zeros((mat_tX.shape[1], 1))
    l, w = logistic_regression_penalized_gradient_descent(y_list[i], mat_tX, initial_w, max_iter, gamma_vec[0], lambda_vec[0])
    print(l)
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)

loop
0.6917728327249799
loop
0.69220057050903
loop
0.6921644909374551


## Generate predictions and save ouput in csv format for submission:

In [38]:
# Data augmentation
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission
mat_tX_test = build_poly(tX_test, 3)
print(mat_tX_test.shape)
y_pred = predict_labels(weights, mat_tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

(568238, 240)


ValueError: shapes (568238,240) and (4,30) not aligned: 240 (dim 1) != 4 (dim 0)

In [41]:
# Separated dataset
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission

tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)
y_pred_list = separated_eval(weights_list, tX_test_list)

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3]))

create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)

In [12]:
# Data augmentation and separated dataset
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission

tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)   #separate dateset

mat_tX_test_list = []

# for i in range(4):
#     print(i)
#     mat_tX_test = build_poly(tX_test_list[i], degree_vec[i])
#     mat_tX_test_list.append(mat_tX_test)

# mat_tX_test_list = build_poly_separated(tX_test_list, 3)            #data augmentation
y_pred_list = separated_eval(weights_list, mat_tX_test_list)        #prediction

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3]))

create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)

0
1
2
3


## Prediction

In [46]:
# Eval

y_pred_list = separated_eval(weights_list, mat_tX_test_list) 

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2]))

OUTPUT_PATH = 'result.csv'
create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)